First make sure frames that have no predictions, empty file, number of frame as title. For files that have a predition the format is as follows: 0 0.2609 0.3657 0.009896 0.01389

In [30]:
import os
import cv2
import numpy as np

In [35]:
def add_missing_frame_files(folder_path):
    """
    Ensures that every frame number from 1 to 750 has a corresponding .txt file in the folder.
    If a .txt file is missing, it creates an empty one.

    Parameters:
        folder_path (str): Path to the folder containing the .txt files.
    """
    try:
        # List all files in the folder
        existing_files = os.listdir(folder_path)

        # Extract numbers from filenames (assume filenames are like '1.txt', '2.txt', etc.)
        existing_frame_numbers = {
            int(file.split('.')[0]) for file in existing_files if file.endswith('.txt') and file.split('.')[0].isdigit()
        }

        # Define the full range of frame numbers
        full_range = set(range(1, 728))

        # Find missing frame numbers
        missing_frames = full_range - existing_frame_numbers

        # Create empty .txt files for missing frames
        for frame in missing_frames:
            empty_file_path = os.path.join(folder_path, f"{frame}.txt")
            with open(empty_file_path, 'w') as empty_file:
                pass  # Create an empty file

        print(f"Added {len(missing_frames)} missing frame files.")

    except Exception as e:
        print(f"An error occurred: {e}")



In [36]:
add_missing_frame_files("C:/Studie/BEP/BEP_Football_tracking/Dataset_ball_eval/sequences/labels/nl_sample")

Added 17 missing frame files.


In [37]:
def create_video_with_annotations(image_folder, annotation_folder, output_video, fps=30):
    """
    Create a video with bounding box visualizations from images and annotations.

    Args:
        image_folder (str): Path to the folder containing images (e.g., 0001.jpg, 0002.jpg, ...).
        annotation_folder (str): Path to the folder containing annotation text files (e.g., 1.txt, 2.txt, ...).
        output_video (str): Path to the output video file (e.g., output.mp4).
        fps (int): Frames per second for the output video.
    """
    import os
    import cv2

    # Get sorted list of image files and annotation files
    image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg')])
    annotation_files = sorted([f for f in os.listdir(annotation_folder) if f.endswith('.txt')],
                               key=lambda x: int(x.split('.')[0]))

    # Ensure image and annotation counts align
    if len(image_files) != len(annotation_files):
        raise ValueError("Number of images and annotations do not match.")

    # Read the first image to get dimensions
    first_image_path = os.path.join(image_folder, image_files[0])
    first_image = cv2.imread(first_image_path)
    height, width, _ = first_image.shape

    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    for image_file, annotation_file in zip(image_files, annotation_files):
        # Read image
        image_path = os.path.join(image_folder, image_file)
        image = cv2.imread(image_path)

        # Read annotation
        annotation_path = os.path.join(annotation_folder, annotation_file)
        if os.path.getsize(annotation_path) > 0:  # Check if file is not empty
            with open(annotation_path, 'r') as f:
                bbox = eval(f.read().strip())  # Safely parse the list

            # Draw bounding box on the image
            x, y, w, h = bbox
            top_left = (x, y)
            bottom_right = (x + w, y + h)
            color = (0, 0, 255)
            thickness = 2
            cv2.rectangle(image, top_left, bottom_right, color, thickness)

        # Add frame to video
        video_writer.write(image)

    # Release the video writer
    video_writer.release()
    print(f"Video saved to {output_video}")

In [38]:
path_to_images = 'C:/Studie/BEP/BEP_Football_tracking/Dataset_ball_eval/sequences/imgs/nl_sample'
path_to_annotations = 'C:/Studie/BEP/BEP_Football_tracking/Dataset_ball_eval/sequences/labels/nl_sample'
output_video = 'C:/Studie/BEP/BEP_Football_tracking/Dataset_ball_eval/sequences/videos/nl_sample.mp4'

create_video_with_annotations(path_to_images, path_to_annotations, output_video)

Video saved to C:/Studie/BEP/BEP_Football_tracking/Dataset_ball_eval/sequences/videos/nl_sample.mp4


Convert